In [ ]:
import datetime
import powerlaw
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from countryinfo import CountryInfo
from rw_data_processing import *
plt.style.use(r"./rw_visualization.mplstyle")

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Color
current_palette = sns.color_palette()
current_palette

# 1. Load Taiwan data and data processing

In [ ]:
# Load data
Taiwan_data_sheet = pd.read_excel('covidtable_taiwan_translated.xlsx', sheet_name=0)
Taiwan_data_sheet.columns = Taiwan_data_sheet.columns.str.strip().str.lower().str.\
    replace(' ', '_').str.replace('(', '').str.replace(')', '')

# Clean data
data_1_to_579 = clean_taiwan_data(Taiwan_data_sheet, 1, 579)

# 2. Extract contact information and construct the contact network


In [ ]:
# Generate undirected contact network that contain all contacts
taiwan_contact_network, contact_type_dict, uninfected_contact_type_dict = generate_taiwan_contact_network(
    data_1_to_579)

In [ ]:
# Generate directed contact network that contain only infected contacts
infection_contact_network, contact_type_dict, uninfected_contact_type_dict = generate_taiwan_infection_contact_network(
    data_1_to_579)

In [ ]:
# Merge the two networks and generate edge list
edge_list = generate_edge_list(
    taiwan_contact_network, infection_contact_network)

In [ ]:
# Save edge list
write_cytoscape_file(edge_list, 'edge_list.csv')

In [ ]:
# Save the edge list related to I269
target_nodes = ['I269', 'I277', 'I284', 'I299']
edge_list_I269 = []
for row in edge_list:
    if (row[0] in target_nodes) or (row[1] in target_nodes):
        edge_list_I269.append(row)

write_cytoscape_file(edge_list_I269, 'edge_list_I269.csv')

# 3. Extract course of disease data and visualize it.

Plot the KM plot to visualize the state transition data and the number of days spent on each transition.


## 3.1 Extract course of disease data

In [ ]:
asymptomatic_to_symptom_days = extract_state_data(
    data_1_to_579, 'infection_date', 'onset_of_symptom', 'recovery')
asymptomatic_to_recover_days = extract_state_data(data_1_to_579, 'infection_date', 'recovery',
                                                  'onset_of_symptom')
symptom_to_icu_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'icu', 'recovery')
symptom_to_recover_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'recovery', 'icu')
icu_to_recover_days = extract_state_data(
    data_1_to_579, 'icu', 'recovery', 'death_date')
icu_to_dead_days = extract_state_data(
    data_1_to_579, 'icu', 'death_date', 'recovery')

asymptomatic_to_confirmed_days = extract_state_data(
    data_1_to_579, 'infection_date', 'confirmed_date', 'onset_of_symptom')
symptomatic_to_cinfirmed_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'confirmed_date', 'recovery')

## 3.2 KM plot

In [ ]:
fig = plt.figure(figsize=(18, 18))
ax1 = fig.add_subplot(331)
_ = state_transition_plot(asymptomatic_to_symptom_days,
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{I}^\mathrm{S}$')
ax1.set_xlabel('')

ax2 = fig.add_subplot(332)
_ = state_transition_plot(asymptomatic_to_recover_days,
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{R}$')
ax2.set_xlabel('')
ax2.set_ylabel('')

ax3 = fig.add_subplot(333)
_ = state_transition_plot(asymptomatic_to_confirmed_days,
                          '$\mathrm{I}^\mathrm{A}$', '$\mathrm{C}$')
ax3.set_xlabel('')
ax3.set_ylabel('')

ax4 = fig.add_subplot(334)
_ = state_transition_plot(
    symptom_to_icu_days, '$\mathrm{I}^\mathrm{S}$', '$\mathrm{I}^\mathrm{C}$')
ax4.set_xlabel('')

ax5 = fig.add_subplot(335)
_ = state_transition_plot(symptom_to_recover_days,
                          '$\mathrm{I}^\mathrm{S}$', '$\mathrm{R}$')
ax5.set_xlabel('')
ax5.set_ylabel('')

ax6 = fig.add_subplot(336)
_ = state_transition_plot(symptomatic_to_cinfirmed_days,
                          '$\mathrm{I}^\mathrm{S}$', '$\mathrm{C}$')
ax6.set_ylabel('')

ax7 = fig.add_subplot(337)
_ = state_transition_plot(
    icu_to_recover_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{R}$')

ax8 = fig.add_subplot(338)
_ = state_transition_plot(
    icu_to_dead_days, '$\mathrm{I}^\mathrm{C}$', '$\mathrm{D}$')
ax8.set_ylabel('')

# plt.savefig('Taiwan_COVID_state_transition_RW2023.pdf')

# 4. Print out course of disease and contact data statistics

## 4.1 Course of disease data statistics

In [ ]:
# 1 to 579 and the case I530 was removed from the Taiwan CDC
total_case_number = len(data_1_to_579)

In [ ]:
number_of_abroad_cases = sum(data_1_to_579['abroad/local'] == 'Abroad')
number_of_local_cases = sum(data_1_to_579['abroad/local'] == 'Local')
number_of_navy_cases = sum(data_1_to_579['abroad/local'] == 'Navy')
number_of_unknown = sum(
    (data_1_to_579['abroad/local'] == 'C') | (data_1_to_579['abroad/local'] == 'X'))
print(
    f'Number of abroad cases: {number_of_abroad_cases}/{total_case_number} ({number_of_abroad_cases/total_case_number*100:.1f}%)')
print(
    f'Number of local cases: {number_of_local_cases}/{total_case_number} ({number_of_local_cases/total_case_number*100:.1f}%)')
print(
    f'Number of navy cases: {number_of_navy_cases}/{total_case_number} ({number_of_navy_cases/total_case_number*100:.1f}%)')
print(
    f'Number of unknown cases: {number_of_unknown}/{total_case_number} ({number_of_unknown/total_case_number*100:.1f}%)')

In [ ]:
number_date_of_traveling = sum(((data_1_to_579['date_of_travelling'] != 'C') &
                                (data_1_to_579['date_of_travelling'] != 'X')) &
                               (data_1_to_579['visited_country/city'] != 'Panshi fast combat support ship'))
number_travel_country = sum((data_1_to_579['visited_country/city'] != 'C') &
                            (data_1_to_579['visited_country/city'] != 'X') &
                            (data_1_to_579['visited_country/city'] != 'Panshi fast combat support ship'))
print(
    f'Number of date of traveling: {number_date_of_traveling}/{number_of_abroad_cases} ({number_date_of_traveling/number_of_abroad_cases*100:.1f}%)')
print(
    f'Number of travel country: {number_travel_country}/{number_of_abroad_cases} ({number_travel_country/number_of_abroad_cases*100:.1f}%)')

In [ ]:
number_of_male = sum(data_1_to_579['gender'] == 'Male')
number_of_female = sum(data_1_to_579['gender'] == 'Female')
number_of_unknown = sum((data_1_to_579['gender'] == 'C') | (
    data_1_to_579['gender'] == 'X'))
print(
    f'Number of male cases: {number_of_male}/{total_case_number} ({number_of_male/total_case_number*100:.1f}%)')
print(
    f'Number of female cases: {number_of_female}/{total_case_number} ({number_of_female/total_case_number*100:.1f}%)')
print(
    f'Number of unknown cases: {number_of_unknown}/{total_case_number} ({number_of_unknown/total_case_number*100:.1f}%)')

In [ ]:
number_of_0_to_9 = sum(data_1_to_579['age'] == '0 to 9')
number_of_10_to_19 = sum(data_1_to_579['age'] == '10 to 19')
number_of_20_to_29 = sum(data_1_to_579['age'] == '20 to 29')
number_of_30_to_39 = sum(data_1_to_579['age'] == '30 to 39')
number_of_40_to_49 = sum(data_1_to_579['age'] == '40 to 49')
number_of_50_to_59 = sum(data_1_to_579['age'] == '50 to 59')
number_of_60_to_69 = sum(data_1_to_579['age'] == '60 to 69')
number_of_70_to_79 = sum(data_1_to_579['age'] == '70 to 79')
number_of_80_to_89 = sum(data_1_to_579['age'] == '80 to 89')
number_of_unknown = sum(
    (data_1_to_579['age'] == 'C') | (data_1_to_579['age'] == 'X'))
print(
    f'Number of 0 to 9 cases: {number_of_0_to_9}/{total_case_number} ({number_of_0_to_9/total_case_number*100:.1f}%)')
print(
    f'Number of 10 to 19 cases: {number_of_10_to_19}/{total_case_number} ({number_of_10_to_19/total_case_number*100:.1f}%)')
print(
    f'Number of 20 to 29 cases: {number_of_20_to_29}/{total_case_number} ({number_of_20_to_29/total_case_number*100:.1f}%)')
print(
    f'Number of 30 to 39 cases: {number_of_30_to_39}/{total_case_number} ({number_of_30_to_39/total_case_number*100:.1f}%)')
print(
    f'Number of 40 to 49 cases: {number_of_40_to_49}/{total_case_number} ({number_of_40_to_49/total_case_number*100:.1f}%)')
print(
    f'Number of 50 to 59 cases: {number_of_50_to_59}/{total_case_number} ({number_of_50_to_59/total_case_number*100:.1f}%)')
print(
    f'Number of 60 to 69 cases: {number_of_60_to_69}/{total_case_number} ({number_of_60_to_69/total_case_number*100:.1f}%)')
print(
    f'Number of 70 to 79 cases: {number_of_70_to_79}/{total_case_number} ({number_of_70_to_79/total_case_number*100:.1f}%)')
print(
    f'Number of 80 to 89 cases: {number_of_80_to_89}/{total_case_number} ({number_of_80_to_89/total_case_number*100:.1f}%)')
print(
    f'Number of unknown cases: {number_of_unknown}/{total_case_number} ({number_of_unknown/total_case_number*100:.1f}%)')

In [ ]:
number_of_asymptomatic = sum((data_1_to_579['infection_date'] != 'C') &
                             (data_1_to_579['infection_date'] != 'X'))
number_of_symptomatic = sum((data_1_to_579['onset_of_symptom'] != 'C') &
                            (data_1_to_579['onset_of_symptom'] != 'X'))
number_of_symptoms = sum((data_1_to_579['symptom'] != 'C')
                         & (data_1_to_579['symptom'] != 'X'))
number_of_confirmed = sum((data_1_to_579['confirmed_date'] != 'C') &
                          (data_1_to_579['confirmed_date'] != 'X'))
number_of_critically_ill = sum((data_1_to_579['icu'] != 'C') &
                               (data_1_to_579['icu'] != 'X'))
number_of_recovered = sum((data_1_to_579['recovery'] != 'C') &
                          (data_1_to_579['recovery'] != 'X'))
number_of_report = sum((data_1_to_579['report_to_cdc'] != 'C') &
                       (data_1_to_579['report_to_cdc'] != 'X'))
number_of_death = sum((data_1_to_579['death_date'] != 'C') &
                       (data_1_to_579['death_date'] != 'X'))
print(
    f'Number of asymptomatic: {number_of_asymptomatic}/{total_case_number} ({number_of_asymptomatic/total_case_number*100:.1f}%)')
print(
    f'Number of symptomatic: {number_of_symptomatic}/{number_of_symptoms} ({number_of_symptomatic/number_of_symptoms*100:.1f}%)')
print(
    f'Number of confirmed: {number_of_confirmed}/{total_case_number} ({number_of_confirmed/total_case_number*100:.1f}%)')
print(f'Number of critically ill: {number_of_critically_ill}')
print(
    f'Number of recovered: {number_of_recovered}/{526} ({number_of_recovered/526*100:.1f}%)')
print(
    f'Number of report: {number_of_report}/{total_case_number} ({number_of_report/total_case_number*100:.1f}%)')
print(
    f'Number of death: {number_of_death}/{total_case_number} ({number_of_death/total_case_number*100:.1f}%)')

## 4.2 Contact data statistics

In [ ]:
def count_clusters(multigraph):
    # Get connected components (clusters)
    clusters = list(nx.connected_components(multigraph))
    num_clusters = len(clusters)
    return num_clusters

In [ ]:
def count_cluster_sizes(multigraph):
    # Get connected components (clusters)
    clusters = list(nx.connected_components(multigraph))

    # Calculate sizes of clusters
    cluster_sizes = [len(cluster) for cluster in clusters]
    # Calculate number of source infected cases
    infected_cases = []
    for cluster in clusters:
        number_of_infected_case = 0
        for node in cluster:
            if isinstance(node, str):
                number_of_infected_case += 1
        infected_cases.append(number_of_infected_case)

    return cluster_sizes, infected_cases

In [ ]:
cluster_num = count_clusters(taiwan_contact_network)
print(f'Number of clusters: {cluster_num}')

In [ ]:
cluster_sizes, infected_cases = count_cluster_sizes(taiwan_contact_network)
print(f'cluster sizes [Min, Mean, Median, Max]: \
    {min(cluster_sizes), np.mean(cluster_sizes), np.median(cluster_sizes), max(cluster_sizes)}')

In [ ]:
direction_edge_number = 0
infection_edge_number = 0
for row in edge_list:
    if row[-1] == True:
        direction_edge_number += 1
    if isinstance(row[0], str) and isinstance(row[1], str):
        infection_edge_number += 1

print(f'Number of nodes: {len(taiwan_contact_network.nodes)}')
print(f'Number of infected nodes: {len(data_1_to_579)}')
print(
    f'Number of uninfected nodes: {len(taiwan_contact_network.nodes) - len(data_1_to_579)}')

print(f'Number of edges: {len(edge_list)}')
print(f'Number of directional edge: {direction_edge_number}')
print(f'Number of infection edge: {infection_edge_number}')
print(
    f'Percentage of direction edge within all edges: {direction_edge_number/len(edge_list)*100}%')
print(
    f'Percentage of direction edge within infection edges: {direction_edge_number/infection_edge_number*100}%')

In [ ]:
# Plot bar char for contact type edges
contact_type_number = max(max(contact_type_dict.values()), max(
    uninfected_contact_type_dict.values()))+1
contact_type_array = np.zeros(contact_type_number)
infection_contact_type_array = np.zeros(contact_type_number)

for row in edge_list:
    i = row[2]
    contact_type_array[i] += 1
    if isinstance(row[0], str) and isinstance(row[1], str):
        infection_contact_type_array[i] += 1

x = np.arange(contact_type_number)
contact_type_x_axis = []
for i in range(contact_type_number):
    try:
        contact_type_x_axis.append(contact_type_dict.inverse[i])
    except:
        contact_type_x_axis.append(uninfected_contact_type_dict.inverse[i])

# Sort based on the contact_type_array
sorted_indices = np.argsort(contact_type_array)
contact_type_array = contact_type_array[sorted_indices]
infection_contact_type_array = infection_contact_type_array[sorted_indices]
contact_type_x_axis = [contact_type_x_axis[i] for i in sorted_indices]

bar_width = 0.35
plt.bar(x-bar_width/2, infection_contact_type_array, width=bar_width, label='Infection edges',
        color=current_palette[0])
plt.bar(x+bar_width/2, contact_type_array, width=bar_width, label='Total edges',
        color=current_palette[1])

plt.yscale('log')
plt.xticks(x, contact_type_x_axis, rotation=45, ha='right')

# plt.xlabel('Contact type')
plt.ylabel('Frequency')
plt.xlim(-0.5, contact_type_number-0.5)

plt.legend()
# plt.savefig('contact_type_bar_RW2023.pdf')
plt.show()

In [ ]:
contact_type_dict = {'grandparent/grandchild':'Family', 'brother/sister':'Family',
                     'coral_princess':'Ship', 'the_same_hotel':'Hotel', 
                     'parent/child':'Family', 'friend':'Friend',
                     'the_same_quarantine_hotel':'Hotel', 'coworker':'Workplace',
                     'the_same_hospital':'Health care', 'couple':'Couple',
                     'live_together':'Household', 'family':'Family',
                     'the_same_car':'Car', 'travel_together':'Travel',
                     'the_same_flight_nearby_seat':'Flight (nearby seat)',
                     'the_same_school':'School', 'the_same_flight':'Flight',
                     'number_of_uninfected_contact_ship':'Ship',
                     'panshi_fast_combat_support_ship':'Ship',
                     'other_unknown_contact':'Municipality'}

In [ ]:
contact_types = np.array(['Family', 'Ship', 'Hotel', 'Friend', 'Workplace',
                          'Health care', 'Couple', 'Household', 'Car', 'Travel', 
                          'Flight (nearby seat)', 'School', 'Flight', 'Municipality'])
mapped_infections = np.zeros(len(contact_types))
mapped_contacts = np.zeros(len(contact_types))
for i, contact_type  in enumerate(contact_type_x_axis):
    mapped_contact_type = contact_type_dict[contact_type]
    infections = infection_contact_type_array[i]
    contacts = contact_type_array[i]
    
    index = np.where(contact_types == mapped_contact_type)[0][0]
    mapped_infections[index] += infections
    mapped_contacts[index] += contacts

if np.sum(mapped_contacts) != np.sum(contact_type_array):
    print('Error: The sum of mapped contacts does not match the sum of contact_type_array')

sorted_indices = np.argsort(mapped_contacts)
mapped_contacts = mapped_contacts[sorted_indices]
mapped_infections = mapped_infections[sorted_indices]
contact_types = contact_types[sorted_indices]

x = np.arange(len(contact_types))
plt.bar(x-bar_width/2, mapped_infections, width=bar_width, label='Infected contacts',
        color=current_palette[0])
plt.bar(x+bar_width/2, mapped_contacts, width=bar_width, label='Total contacts',
        color=current_palette[1])
plt.yscale('log')
plt.xticks(x, contact_types, rotation=45, ha='right')

plt.ylabel('Frequency')
plt.xlim(-0.5, x[-1]+0.5)

plt.legend(loc='best')
# plt.savefig('contact_type_bar_RW2023.pdf')
plt.show()

In [ ]:
### Calculate the degree of the network
degrees = [degree for node, degree in taiwan_contact_network.degree()]

fit = powerlaw.Fit(degrees, discrete=True, xmin=1, estimate_discrete=True)
### Fit the powerlaw distribution and calculate the goodness of fit
fit_powerlaw = fit.power_law
print(f'Power Law alpha = {fit_powerlaw.alpha}, sigma = {fit_powerlaw.sigma}')


### Fit the lognormal distribution and calculate the goodness of fit
fit_lognormal = fit.lognormal
print(f'Lognormal mu = {fit_lognormal.mu}, sigma = {fit_lognormal.sigma}')


## Fit the exponential distribution and calculate the goodness of fit
fit_exponential = fit.exponential
print(f'Exponential lambda = {fit_exponential.Lambda}\n')

print(f'Power Law goodness of fit = {fit_powerlaw.KS()}')
print(f'Lognormal goodness of fit = {fit_lognormal.KS()}')
print(f'Exponential goodness of fit = {fit_exponential.KS()}\n')

print(f"Compare powerlaw & lognormal: {fit.distribution_compare('power_law', 'lognormal')}")
print(f"Compare powerlaw & exponential: {fit.distribution_compare('power_law', 'exponential')}")
print(f"Compare lognormal & exponential: {fit.distribution_compare('lognormal', 'exponential')}\n")

### Visualize the degree distribution and the fitted powerlaw, lognormal and exponential distributions
fig, ax = plt.subplots()
fit.plot_ccdf(color=current_palette[0], linewidth=2, ax=ax)
fit_powerlaw.plot_ccdf(color=current_palette[1], linewidth=1, linestyle='--', ax=ax)
fit_lognormal.plot_ccdf(color=current_palette[2], linewidth=1, linestyle='--', ax=ax)
fit_exponential.plot_ccdf(color=current_palette[3], linewidth=1, linestyle='--', ax=ax)

plt.xlabel('Degree')
plt.ylabel('Complementary cumulative distribution function (CCDF)')
plt.legend(['Data', 'Power Law', 'Lognormal', 'Exponential'], loc='best')
plt.show()

fig, ax = plt.subplots()
fit.plot_cdf(color=current_palette[0], linewidth=2, ax=ax)
fit_powerlaw.plot_cdf(color=current_palette[1], linewidth=1, linestyle='--', ax=ax)
fit_lognormal.plot_cdf(color=current_palette[2], linewidth=1, linestyle='--', ax=ax)
fit_exponential.plot_cdf(color=current_palette[3], linewidth=1, linestyle='--', ax=ax)

plt.xlabel('Degree')
plt.ylabel('Cumulative distribution function (CDF)')
plt.legend(['Data', 'Power Law', 'Lognormal', 'Exponential'], loc='best')
plt.savefig('degree_distribution_cdf_RW2023.pdf')
plt.show()

# 5. Reproduce statistics

## 5.1 Liu2020_Analysis_of_Imported_Cases_of_COVID-19_in_Taiwan_A_Nationwide_Study

Note: 

1. "We analysed the characteristics, infection source, symptom presentation,
and route of identiﬁcation of the 321 imported cases that were identiﬁed from 21 January to 6 April 2020."

2. "There was a cumulative total of 373 confirmed cases of COVID-19 in Taiwan from 21 January to 
6 April 2020, 321 (86.1%) of which were imported."

3. "Of the imported cases, 96.6\% were Taiwanese and 53% were femail."

In [ ]:
# Load data
data_1_to_373 = clean_taiwan_data(Taiwan_data_sheet, 1, 373)

data_liu_abroad = data_1_to_373[data_1_to_373['abroad/local']=='Abroad']
print(f'Number of abroad cases: {len(data_liu_abroad)}')
data_liu_local = data_1_to_373[data_1_to_373['abroad/local']=='Local']
print(f'Number of local cases: {len(data_liu_local)}')

### 5.1.1 Figure 1 stacked bar chart

In [ ]:
for i in data_liu_abroad['age'].unique():
    percentage = (sum(data_liu_abroad['age']==i))/321*100
    print(f"{i}: {percentage:.2f}%")


In [ ]:
data_liu_abroad_male = data_liu_abroad[data_liu_abroad['gender']=='Male']
data_liu_abroad_female = data_liu_abroad[data_liu_abroad['gender']=='Female']
print(f'Percentage of male cases: {len(data_liu_abroad_male)/321*100:.2f}%')
print(f'Percentage of female cases: {len(data_liu_abroad_female)/321*100:.2f}%')
male_age_list = []
female_age_list = []
age_index = np.sort(data_liu_abroad_male['age'].unique())
for i in age_index:
    num_age = sum(data_liu_abroad_male['age']==i)
    male_age_list.append(num_age)
    num_age = sum(data_liu_abroad_female['age']==i)
    female_age_list.append(num_age)

In [ ]:
# Stacked bar chart
plt.figure(figsize=(7, 8))
plt.bar(age_index, female_age_list, bottom=male_age_list, label='Female')
plt.bar(age_index, male_age_list, label='Male')
plt.xticks(range(len(age_index)), ['<10', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89'])
plt.legend()
plt.xlabel('Age (years)')
plt.ylabel('Number of cases')
plt.ylim([0, 130])
plt.xlim([-1, 9])
# plt.savefig('Reproduce_liu2020_imported_fig1_RW2023.pdf')

### 5.1.2 Figure 2: Date of arrival

In [ ]:
country_list = data_liu_abroad['infection_source'].to_list()
arrival_date_list = data_liu_abroad['date_of_arrival'].to_list()
for i, time in enumerate(arrival_date_list):
    if type(time) != str:
        arrival_date_list[i] = time.strftime('%Y-%m-%d')

In [ ]:
def get_continent_liu2020(country_list):
    continent_list = []
    for country in country_list:
        if country == 'China':
            continent_list.append('China')
        elif country == 'Europe':
            continent_list.append('Europe')
        elif country == 'South America':
            continent_list.append('South America')
        elif country == 'Southeast Asia':
            continent_list.append('East & South Asia')
        elif country == 'North America':
            continent_list.append('North America')
        else:
            try:
                country_info = CountryInfo(country)
                continent = country_info.subregion()
                if continent == 'South-Eastern Asia' or continent == 'Eastern Asia':
                    continent = 'East & South Asia'
                elif continent in ['Northern Europe', 'Southern Europe', 'Western Europe', 'Eastern Europe']:
                    continent = 'Europe'
                elif continent == 'Australia and New Zealand':
                    continent = 'Oceania'
                elif continent in ['Central America', 'South America']:
                    continent = 'South America'
                elif continent in ['Northern Africa', 'Southern Africa', 'Western Asia']:
                    continent = 'Middle East & Africa'
                elif continent == 'Northern America':
                    continent = 'North America'
                continent_list.append(continent)
            except Exception as e:
                print(f'Warring, can not find continent for {country}')
                continent_list.append('Unknown')
            
    return(continent_list)


In [ ]:
continent_list = get_continent_liu2020(country_list)

In [ ]:
# Plot stacked bar chart
fig, ax = plt.subplots(figsize=(10, 6))

# Group data by date and continent
grouped_data = pd.DataFrame({'date': arrival_date_list, 'continent': continent_list})
grouped_data = grouped_data.groupby(['date', 'continent']).size().reset_index(name='count')

# Pivot data to create stacked bar chart
pivot_data = grouped_data.pivot(index='date', columns='continent', values='count')
pivot_data = pivot_data.fillna(0)

# Define order of stacked bars
stack_order = ['South America', 'Oceania', 'North America', 'Middle East & Africa', 'Europe', 'China', 'East & South Asia', 'Unknown']

# Plot stacked bar chart
pivot_data[stack_order].plot(kind='bar', stacked=True, ax=ax, color=[current_palette[4], '#a6bddb', current_palette[0], current_palette[3], current_palette[1], current_palette[2], current_palette[5], '#7f7f7f'])

# Set x-axis label
ax.set_xlabel('Date of arrival')

# Set y-axis label
ax.set_ylabel('Number of cases')

ax.set_xlim([-0.5, 42.5])
ax.set_ylim([0, 33])

# Remove word "continent" from legend
handles, labels = ax.get_legend_handles_labels()
labels = [label.replace('continent', '') for label in labels]
ax.legend(handles, labels, loc='best')

# Show plot
plt.show()
# plt.savefig('Reproduce_liu2020_imported_fig2_RW2023.pdf')



### 5.1.3 Table 2

In [ ]:
symptom_onsets = data_liu_abroad['onset_of_symptom'].to_list()
liu_id = data_liu_abroad['id'].to_list()

In [ ]:
symptoms = data_liu_abroad['symptom']
unique_symptoms = np.array([])
for i, symptom in enumerate(symptoms):
    symptom_tmp = np.array([s.lower() for s in symptom.split(', ')])
    unique_symptoms = np.hstack([unique_symptoms, symptom_tmp])

unique_symptoms = np.unique(unique_symptoms)
print(unique_symptoms)

In [ ]:
liu_symptom_list = np.array(['fever', 'chills', 'malaise', 'myalgia or arthralgia', 'cough',
                'sore throat', 'rhinorrhoea, sneezing, nasal stiffiness',
                'chest tightness or pain', 'dyspnea', 'loss of smell or taste', 
                'headache', 'dizziness', 'diarrhoea', 'abdominal pain', 'nausea or vomiting',
                'itching, congestion, pain in the eyes', 'no symptoms'])
liu_symptom_dict = {symptom: i for i, symptom in enumerate(liu_symptom_list)}
print(liu_symptom_dict)
print(np.sort(liu_symptom_list))

my_symptom_to_liu_symptom_dict = \
   {'abdominal pain': 'abdominal pain', 'chest pain': 'chest tightness or pain',
    'chills or rigours': 'chills', 
    'congestion or runny nose': 'rhinorrhoea, sneezing, nasal stiffiness',
    'cough': 'cough', 'diarrhea': 'diarrhoea', 'dizziness': 'dizziness',
    'dyspnea': 'dyspnea', 'fever': 'fever', 'general fatigue': 'malaise',
    'headache': 'headache', 'loss of smell': 'loss of smell or taste', 
    'loss of taste': 'loss of smell or taste', 'lymphadenopathy': 'Others', 
    'muscle or body pain': 'myalgia or arthralgia', 'nausea': 'nausea or vomiting',
    'pneumonia': 'Others', 'rds': 'Others', 
    'red or irritated eyes': 'itching, congestion, pain in the eyes',
    'respiratory infectious disease': 'rhinorrhoea, sneezing, nasal stiffiness',
    'running nose': 'rhinorrhoea, sneezing, nasal stiffiness',
    'sneeze': 'rhinorrhoea, sneezing, nasal stiffiness',
    'sore throat': 'sore throat', 'sweating': 'Others', 
    'upper respiratory symptoms': 'rhinorrhoea, sneezing, nasal stiffiness',
    'vomiting': 'nausea or vomiting', 'x': 'None'}

In [ ]:
# Count number of symptom 
count_liu_symptom_list = np.zeros(len(liu_symptom_list))
for i, symptom in enumerate(symptoms):
    count_list_tmp = np.zeros(len(liu_symptom_list))
    for symptom_tmp in symptom.split(', '):
        symptom_tmp = symptom_tmp.lower()
        symptom_tmp_map = my_symptom_to_liu_symptom_dict[symptom_tmp]
        if (symptom_tmp_map != 'None') and (symptom_tmp_map != 'Others'):
            count_list_tmp[liu_symptom_dict[symptom_tmp_map]] = 1
        elif (symptom_tmp_map == 'None') and (symptom_tmp_map != 'Others'):
            if symptom_onsets[i] == 'X':
                count_list_tmp[liu_symptom_dict['no symptoms']] = 1
    count_liu_symptom_list += count_list_tmp
        

In [ ]:
liu_symptom_count_table = pd.DataFrame({'Symptom': liu_symptom_list, 'Count': np.int32(count_liu_symptom_list), 'Percent': np.round((count_liu_symptom_list/321)*1000)/10})
try:
    print(liu_symptom_count_table)
except:
    print("Error: Could not print liu_symptom_count_table")


### 5.1.4 Figure 3

In [ ]:
days_from_arrival_to_symptom = np.array([])
for i, symptom_onset in enumerate(symptom_onsets):
    if isinstance(symptom_onset, datetime.datetime):
        days_from_arrival_to_symptom = \
            np.append(days_from_arrival_to_symptom, 
                      (symptom_onset - datetime.datetime.strptime(arrival_date_list[i], '%Y-%m-%d')).days)
    else:
        # days_from_arrival_to_symptom = np.append(days_from_arrival_to_symptom, np.nan)
        pass


In [ ]:
# Count how many cases happened each day in days_from_arrival_to_symptom
counts, bins = np.histogram(days_from_arrival_to_symptom, bins=np.arange(np.min(days_from_arrival_to_symptom), np.max(days_from_arrival_to_symptom)))

# Plot a bar chart ranging from -30 to 13
plt.bar(bins[:-1], counts, width=0.8, color=current_palette[0], edgecolor='grey')
plt.bar(bins[0:31], counts[0:31], width=0.8, color='lightblue', edgecolor='grey')
plt.xlabel('Days from arrival to symptom onset')
plt.ylabel('Number of cases')
plt.xlim([-31, 19])
_ = plt.xticks(np.arange(-30, 18, 5))

### 5.1.5 Figure 4

In [ ]:
data_liu_abroad['way_of_discover'].unique()

In [ ]:
discovery_dict = {'self-quarantine': 'home quarantine', 'airport quarantine': 'airport screening',
                  'contact inspection': 'contact tracing and testing',
                  'self-medication': 'hospital notification', 
                  'X': 'contact tracing and testing', 
                  'health unit arranges for medical treatment': 'hospital notification',
                  'hospital quarantine': 'hospital notification',
                  'quarantine': 'hospital notification'}
way_of_discover_symptom = np.array([])
way_of_discover_asymptom = np.array([])
for i, way_of_discover in enumerate(data_liu_abroad['way_of_discover']):
    way_of_discover_liu = discovery_dict[way_of_discover]
    symptom_onset_date = data_liu_abroad['onset_of_symptom'].iloc[i]
    arrival_date = data_liu_abroad['date_of_arrival'].iloc[i]
    try:
        days = (symptom_onset_date-arrival_date).days
        if days <= 0: # symptom before arrival
            way_of_discover_symptom = np.append(way_of_discover_symptom, way_of_discover_liu)
        else:
            way_of_discover_asymptom = np.append(way_of_discover_asymptom, way_of_discover_liu)
    except:
        pass

In [ ]:
values, counts = np.unique(way_of_discover_symptom, return_counts=True)
plt.bar(0, counts[values=='hospital notification'], color=current_palette[4], width=0.2)
plt.bar(0, counts[values=='home quarantine'], bottom=counts[values=='hospital notification'], color=current_palette[3], width=0.2)
plt.bar(0, counts[values=='contact tracing and testing'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine'], color=current_palette[1], width=0.2)
plt.bar(0, counts[values=='airport screening'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine']+counts[values=='contact tracing and testing'], color=current_palette[0], width=0.2)

values, counts = np.unique(way_of_discover_asymptom, return_counts=True)
plt.bar(1, counts[values=='hospital notification'], color=current_palette[4], label='Hospital notification', width=0.2)
plt.bar(1, counts[values=='home quarantine'], bottom=counts[values=='hospital notification'], color=current_palette[3], label='Home quarantine', width=0.2)
plt.bar(1, counts[values=='contact tracing and testing'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine'], color=current_palette[1], label='Contact tracing and testing', width=0.2)
plt.bar(1, counts[values=='airport screening'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine']+counts[values=='contact tracing and testing'], color=current_palette[0], label='Airport screening', width=0.2)

plt.xticks([0, 1], ['Symptom developed before arrival', 'Asymptomatic on arrival'])
plt.ylabel('Number of cases')
plt.legend(loc='best')
plt.xlim([-0.5, 1.5])
plt.ylim([0, 210])

### 5.1.5 Combine the four plot above to a single big plot

In [ ]:
country_list = data_liu_abroad['infection_source'].to_list()
arrival_date_list = data_liu_abroad['date_of_arrival'].to_list()
for i, time in enumerate(arrival_date_list):
    if type(time) != str:
        arrival_date_list[i] = time.strftime('%m/%d')

In [ ]:
data_liu_abroad_male = data_liu_abroad[data_liu_abroad['gender']=='Male']
data_liu_abroad_female = data_liu_abroad[data_liu_abroad['gender']=='Female']
print(f'Percentage of male cases: {len(data_liu_abroad_male)/321*100:.2f}%')
print(f'Percentage of female cases: {len(data_liu_abroad_female)/321*100:.2f}%')
male_age_list = []
female_age_list = []
age_index = np.sort(data_liu_abroad_male['age'].unique())
for i in age_index:
    num_age = sum(data_liu_abroad_male['age']==i)
    male_age_list.append(num_age)
    num_age = sum(data_liu_abroad_female['age']==i)
    female_age_list.append(num_age)

In [ ]:
fontsize = 18
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 19))
ax1.bar(age_index, female_age_list, bottom=male_age_list, label='Female')
ax1.bar(age_index, male_age_list, label='Male')
ax1.set_xticks(range(len(age_index)))
ax1.set_xticks([0, 2, 4, 6, 8])
ax1.set_xticklabels(['<10', '20-29', '40-49', '60-69', '80-89'], fontsize=fontsize)
ax1.legend(fontsize=fontsize)
ax1.set_xlabel('Age (years)', fontsize=fontsize)
ax1.set_ylabel('Number of cases', fontsize=fontsize)
ax1.set_ylim([0, 130])
ax1.set_xlim([-1, 9])
ax1.text(-0.1, 1.1, 'A', transform=ax1.transAxes, fontsize=fontsize, fontweight='bold', va='top', ha='right')
ax1.tick_params(axis='y', labelsize=fontsize)

# Group data by date and continent

grouped_data = pd.DataFrame({'date': arrival_date_list, 'continent': continent_list})
grouped_data = grouped_data.groupby(['date', 'continent']).size().reset_index(name='count')

# Pivot data to create stacked bar chart
pivot_data = grouped_data.pivot(index='date', columns='continent', values='count')
pivot_data = pivot_data.fillna(0)

# Define order of stacked bars
stack_order = ['South America', 'Oceania', 'North America', 'Middle East & Africa', 'Europe', 'China', 'East & South Asia', 'Unknown']

# Plot stacked bar chart
pivot_data[stack_order].plot(kind='bar', stacked=True, ax=ax2, color=[current_palette[4], '#a6bddb', current_palette[0], current_palette[3], current_palette[1], current_palette[2], current_palette[5], '#7f7f7f'])

# Set x-axis label
ax2.set_xlabel('Date of arrival', fontsize=fontsize)
ax2.set_xticks(np.arange(0, len(arrival_date_list), 7))
# ax2.set_xticklabels(arrival_date_list[::7], rotation=20, ha='right')
ax2.set_xticklabels(arrival_date_list[::7], rotation=0, fontsize=fontsize)

# Set y-axis label
ax2.set_ylabel('Number of cases', fontsize=fontsize)
ax2.text(-0.1, 1.1, 'B', transform=ax2.transAxes, fontsize=fontsize, fontweight='bold', va='top', ha='right')
ax2.tick_params(axis='y', labelsize=fontsize)

ax2.set_xlim([-0.5, 42.5])
ax2.set_ylim([0, 33])

# Remove word "continent" from legend
handles, labels = ax2.get_legend_handles_labels()
labels = [label.replace('continent', '') for label in labels]
ax2.legend(handles, labels, loc='best', fontsize=fontsize)

ax3.plot([1, 2, 3], [4, 5, 6])
counts, bins = np.histogram(days_from_arrival_to_symptom, bins=np.arange(np.min(days_from_arrival_to_symptom), np.max(days_from_arrival_to_symptom)))
ax3.bar(bins[:-1], counts, width=0.8, color=current_palette[0], edgecolor='grey')
ax3.bar(bins[0:31], counts[0:31], width=0.8, color='lightblue', edgecolor='grey')
ax3.set_xlabel('Days from arrival to symptom onset', fontsize=fontsize)
ax3.set_ylabel('Number of cases', fontsize=fontsize)
ax3.set_xlim([-31, 19])
ax3.set_xticks(np.arange(-30, 18, 5))
ax3.text(-0.1, 1.1, 'C', transform=ax3.transAxes, fontsize=fontsize, fontweight='bold', va='top', ha='right')
ax3.tick_params(axis='both', labelsize=fontsize)


values, counts = np.unique(way_of_discover_symptom, return_counts=True)
ax4.bar(0, counts[values=='hospital notification'], color=current_palette[4], width=0.2)
ax4.bar(0, counts[values=='home quarantine'], bottom=counts[values=='hospital notification'], color=current_palette[3], width=0.2)
ax4.bar(0, counts[values=='contact tracing and testing'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine'], color=current_palette[1], width=0.2)
ax4.bar(0, counts[values=='airport screening'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine']+counts[values=='contact tracing and testing'], color=current_palette[0], width=0.2)

values, counts = np.unique(way_of_discover_asymptom, return_counts=True)
ax4.bar(1, counts[values=='hospital notification'], color=current_palette[4], label='Hospital notification', width=0.2)
ax4.bar(1, counts[values=='home quarantine'], bottom=counts[values=='hospital notification'], color=current_palette[3], label='Home quarantine', width=0.2)
ax4.bar(1, counts[values=='contact tracing and testing'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine'], color=current_palette[1], label='Contact tracing and testing', width=0.2)
ax4.bar(1, counts[values=='airport screening'], bottom=counts[values=='hospital notification']+counts[values=='home quarantine']+counts[values=='contact tracing and testing'], color=current_palette[0], label='Airport screening', width=0.2)

ax4.set_xticks([0, 1])
ax4.set_xticklabels(['Symptom developed before arrival', 'Asymptomatic on arrival'], fontsize=fontsize)
ax4.set_ylabel('Number of cases', fontsize=fontsize)
ax4.legend(loc='best', fontsize=fontsize)
ax4.set_xlim([-0.5, 1.5])
ax4.set_ylim([0, 220])
ax4.text(-0.1, 1.1, 'D', transform=ax4.transAxes, fontsize=fontsize, fontweight='bold', va='top', ha='right')
ax4.tick_params(axis='both', labelsize=fontsize)

# plt.savefig('Reproduce_liu2020_RW2023.pdf')